In [1]:
import os

In [2]:
%pwd

'd:\\project\\Predict-Lung-Disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\Predict-Lung-Disease'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import gdown
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            gdown.download(self.config.source_URL, self.config.local_data_file, quiet=False)
            logger.info(f"{self.config.local_data_file} downloaded successfully!")
        else:
            logger.info(f"fFile already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-06 15:09:30,459: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-06 15:09:30,459: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-06 15:09:30,459: INFO: common: creating directory at: artifacts]
[2024-10-06 15:09:30,459: INFO: common: creating directory at: artifacts/data_ingestion]


Downloading...
From (original): https://drive.google.com/uc?id=1vgPKQAoZMml4BAel5fySYYTndv9siB1l
From (redirected): https://drive.google.com/uc?id=1vgPKQAoZMml4BAel5fySYYTndv9siB1l&confirm=t&uuid=0abbbe8e-8358-4537-a8c9-993888d88399
To: d:\project\Predict-Lung-Disease\artifacts\data_ingestion\data.zip
100%|██████████| 36.5M/36.5M [00:03<00:00, 9.96MB/s]

[2024-10-06 15:09:38,729: INFO: 3311954803: artifacts/data_ingestion/data.zip downloaded successfully!]
